In [13]:
import numpy as np
import scipy.sparse as sp
import matplotlib.pyplot as plt
import json

## allow the import of the MovieData class
import sys
import os
path = os.path.abspath(os.getcwd()) + '/../data_load'
sys.path.insert(0, path)

from movie_data import MovieData

%matplotlib inline

In [18]:
moviedata = MovieData(min_genre_frequency=0.05)
print(moviedata.genre_labels)

['Action' 'Adventure' 'Comedy' 'Crime' 'Drama' 'Romance' 'Thriller']


In [34]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

vectorizer = TfidfVectorizer(
    max_df=0.7, 
    analyzer='word',
    ngram_range=(1, 1),
    max_features=None,
    stop_words='english')

X = vectorizer.fit_transform(moviedata.plots)
# vectorizer.get_feature_names()

In [35]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, 
    moviedata.one_hot_genres, 
    test_size=0.4, 
    random_state=42
)

In [36]:
X.shape

(5029, 23699)

In [37]:
for i in range(len(moviedata.genre_labels)):
    coeffs = np.linalg.lstsq(X_train, y_train[:, i])[0]
    

LinAlgError: 0-dimensional array given. Array must be two-dimensional